# Orders

✏️ **Exercise**

Today, we will investigate the **orders**, and their associated review score.

For that purpose, we will create one single data table containing **all our orders with some engineered statistics for them as additional columns.**

👉 Our goal is to create a DataFrame with the following features:

*- it will bcome in very handy for our modeling phase -*


| feature_name 	| type 	| description 	|
|:---	|:---:	|:---	|
| `order_id` 	| str 	| the id of the order 	|
| `wait_time` 	| float 	| the number of days between order_date and delivered_date 	|
| `expected_wait_time` 	| float 	| the number of days between order_date and estimated_delivery_date 	|
| `delay_vs_expected` 	| float 	| if the actual delivery date is later than the estimated delivery date, returns the number of days between the two dates, otherwise return 0 	|
| `order_status` 	| str 	| the status of the order 	|
| `dim_is_five_star` 	| int 	| 1 if the order received a five-star review, 0 otherwise 	|
| `dim_is_one_star` 	| int 	| 1 if the order received a one_star, 0 otherwise 	|
| `review_score` 	| int 	| from 1 to 5 	|
| `number_of_products` 	| int 	| number of products that the order contains 	|
| `number_of_sellers` 	| int 	| number of sellers involved in the order 	|
| `price` 	| float 	| total price of the order paid by customer 	|
| `freight_value` 	| float 	| value of the freight paid by customer 	|
| `distance_customer_seller` 	| float 	| the distance in km between customer and seller (optional) 	|  
  
⚠️ We also want to filter out "non-delivered" orders, unless explicitly specified, otherwise we cannot compute the potential delays.

❓ **Your challenge**: 

- Implement each feature as a separate method within the `Order` class available at `olist/order.py`
- Then, create a method `get_training_data()` that returns the complete DataFrame.

💡 Suggested methodology:
- Use the notebook below to write and test your code step-by-step first
- Then copy the code into `order.py` once you are certain of your code logic
- Focus on the data manipulation logic now, we will analyse the dataset visually in the next challenges

🔥 Notebook best practices (must-read) 👇

<details>
    <summary> - <i>click here</i> - </summary>

From now on, exploratory notebooks are going to become pretty long, and we strongly advise you to follow these notebook principles:
- Code your logic so that your Notebook can always be ran from top to bottom without crashing (Cell --> Run All)
- Name your variables carefully 
- Use dummy names such as `tmp` or `_` for intermediary steps when you know you won't need them for long
- Clear your code and merge cells when relevant to minimize Notebook size (`Shift-M`)
- Hide your cell output if you don't need to see it anymore (double-click on the red `Out[]:` section to the left of your cell).
- Make heavy use of jupyter nbextention `Collapsable Headings` and `Table of Content` (call a TA if you can't find them)
- Use the following shortcuts 
    - `a` to insert a cell above
    - `b` to insert a cell below
    - `dd` to delete a cell
    - `esc` and `arrows` to move between cells
    - `Shift-Enter` to execute cell and move focus to the next one
    - use `Shift + Tab` when you are between method brackets e.g. `groupby()` to get the docs! Repeat a few times to open it permanently

</details>





In [2]:
# Auto reload imported module everytime a jupyter cell is executed (handy for olist.order.py updates)
%load_ext autoreload
%autoreload 2

In [3]:
# Import usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
# Import olist data
from olist.data import Olist
olist = Olist()
data = olist.get_data()
matching_table = olist.get_matching_table()

In [5]:
data

{'sellers':                              seller_id  seller_zip_code_prefix  \
 0     3442f8959a84dea7ee197c632cb2df15                   13023   
 1     d1b65fc7debc3361ea86b5f14c68d2e2                   13844   
 2     ce3ad9de960102d0677a81f5d0bb7b2d                   20031   
 3     c0f3eea2e14555b6faeea3dd58c1b1c3                    4195   
 4     51a04a8a6bdcb23deccc82b0b80742cf                   12914   
 ...                                ...                     ...   
 3090  98dddbc4601dd4443ca174359b237166                   87111   
 3091  f8201cab383e484733266d1906e2fdfa                   88137   
 3092  74871d19219c7d518d0090283e03c137                    4650   
 3093  e603cf3fec55f8697c9059638d6c8eb5                   96080   
 3094  9e25199f6ef7e7c347120ff175652c3b                   12051   
 
             seller_city seller_state  
 0              campinas           SP  
 1            mogi guacu           SP  
 2        rio de janeiro           RJ  
 3             sao paul

## Code `order.py`

In [11]:
orders = data['orders']
row[orders]

NameError: name 'row' is not defined

In [5]:
orders = data['orders'].copy() # good practice to be sure not to modify your `data` variable

In [6]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


### `get_wait_time`
    Return a Dataframe with:
           order_id, wait_time, expected_wait_time, delay_vs_expected, order_status

*- Hints -*
- Don't forget to convert dates from "string" type to "pandas.datetime' using [`pandas.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)
- Take time to understand what python [`datetime`](https://docs.python.org/3/library/datetime.html) objects are 

In [7]:
# We give you the pseudo-code below for this first method:
# Inspect orders datarame
orders.shape



(99441, 8)

In [8]:
orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [9]:
orders.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [10]:
# Filter dataframe on delivered orders
orders.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [11]:
# Filter dataframe on delivered orders
orders["order_delivered_carrier_date"]

0        2017-10-04 19:55:00
1        2018-07-26 14:31:00
2        2018-08-08 13:50:00
3        2017-11-22 13:39:59
4        2018-02-14 19:46:34
                ...         
99436    2017-03-10 11:18:03
99437    2018-02-07 23:22:42
99438    2017-08-28 20:52:26
99439    2018-01-12 15:35:03
99440    2018-03-09 22:11:59
Name: order_delivered_carrier_date, Length: 99441, dtype: object

In [12]:
# date_columns=orders.loc[:,"order_purchase_timestamp":"order_estimated_delivery_date"]
# date_columns


In [13]:
        # data = {}
        # for k, f in zip(key_names, file_names):
        #     data[k] = pd.read_csv(os.path.join(csv_path, f))
        # return data

In [14]:
# handle datetime
lis=["order_purchase_timestamp","order_approved_at","order_delivered_carrier_date","order_delivered_customer_date","order_estimated_delivery_date"]

for i in lis:
    orders[i]=pd.to_datetime(orders[i],format="%Y-%m-%d")
    orders[i]


In [15]:
orders.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [16]:
# orders["order_delivered_carrier_date"]=
# pd.to_datetime\(orders["order_delivered_carrier_date"],format="%Y-%m-%d")

In [17]:
orders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [18]:
orders.dropna(how="any")

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15


In [19]:
# compute wait time


In [20]:
orders["wait_time"]=orders["order_delivered_customer_date"] - orders["order_purchase_timestamp"]

In [21]:
# compute expected wait time
orders["expected_wait_time"]=orders["order_estimated_delivery_date"]-orders["order_purchase_timestamp"]

In [22]:
# compute delay vs expected - carefully handles "negative" delays
orders["delay_vs_expected"]=(orders["order_delivered_customer_date"]-orders["order_estimated_delivery_date"]).dt.days
orders["delay_vs_expected"]

0        -8.0
1        -6.0
2       -18.0
3       -13.0
4       -10.0
         ... 
99436   -11.0
99437    -2.0
99438    -6.0
99439   -21.0
99440   -18.0
Name: delay_vs_expected, Length: 99441, dtype: float64

In [23]:
def handle_delay(d):
    if d < 0:
        return 0
    else:
        return d

In [24]:
orders["delay_vs_expected"]=orders["delay_vs_expected"].apply(handle_delay)
orders["delay_vs_expected"]


0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99436    0.0
99437    0.0
99438    0.0
99439    0.0
99440    0.0
Name: delay_vs_expected, Length: 99441, dtype: float64

In [25]:
orders[orders["delay_vs_expected"]>0]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,wait_time,expected_wait_time,delay_vs_expected
20,203096f03d82e0dffbc41ebc2e2bcfb7,d2b091571da224a1b36412c18bc3bbfe,delivered,2017-09-18 14:31:30,2017-09-19 04:04:09,2017-10-06 17:50:03,2017-10-09 22:23:46,2017-09-28,21 days 07:52:16,9 days 09:28:30,11.0
25,fbf9ac61453ac646ce8ad9783d7d0af6,3a874b4d4c4b6543206ff5d89287f0c3,delivered,2018-02-20 23:46:53,2018-02-22 02:30:46,2018-02-26 22:25:22,2018-03-21 22:03:54,2018-03-12,28 days 22:17:01,19 days 00:13:07,9.0
41,6ea2f835b4556291ffdc53fa0b3b95e8,c7340080e394356141681bd4c9b8fe31,delivered,2017-11-24 21:27:48,2017-11-25 00:21:09,2017-12-13 21:14:05,2017-12-28 18:59:23,2017-12-21,33 days 21:31:35,26 days 02:32:12,7.0
57,66e4624ae69e7dc89bd50222b59f581f,684fa6da5134b9e4dab731e00011712d,delivered,2018-03-09 14:50:15,2018-03-09 15:40:39,2018-03-15 00:31:19,2018-04-03 13:28:46,2018-04-02,24 days 22:38:31,23 days 09:09:45,1.0
58,a685d016c8a26f71a0bb67821070e398,911e4c37f5cafe1604fe6767034bf1ae,delivered,2017-03-13 18:14:36,2017-03-13 18:14:36,2017-03-22 14:03:09,2017-04-06 13:37:16,2017-03-30,23 days 19:22:40,16 days 05:45:24,7.0
...,...,...,...,...,...,...,...,...,...,...,...
99396,b3112ca67f3afd4e20cf2ee91fc4f804,6f83c71b6c044fb156d697d4130fe9b5,delivered,2018-08-02 22:46:54,2018-08-02 23:04:06,2018-08-15 17:42:00,2018-08-21 00:03:26,2018-08-16,18 days 01:16:32,13 days 01:13:06,5.0
99397,0fa1fab1d7c1211c824596ed5e111e3c,7f3bd6c94d2daf7b6462d1a894a775b4,delivered,2018-03-13 21:48:57,2018-03-13 22:40:28,2018-03-14 19:27:23,2018-04-05 19:59:49,2018-04-02,22 days 22:10:52,19 days 02:11:03,3.0
99423,38e9133ce29f6bbe35aed9c3863dce01,ad312389a098ceff46ce92c4595c06d0,delivered,2017-10-12 20:54:11,2017-10-14 03:28:24,2017-10-17 17:04:42,2017-11-21 17:06:59,2017-10-31,39 days 20:12:48,18 days 03:05:49,21.0
99425,d692ef54145c9cb3322ec2e5508aa3f4,82ddfcf9438b0cd1117b55ac33184df8,delivered,2018-03-21 19:47:18,2018-03-21 20:05:26,2018-03-22 21:11:58,2018-04-11 00:48:31,2018-04-09,20 days 05:01:13,18 days 04:12:42,2.0


In [26]:
orders[["order_id", "wait_time", "expected_wait_time",
                            "delay_vs_expected", "order_status"]]

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8 days 10:28:40,15 days 13:03:27,0.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13 days 18:46:08,19 days 03:18:23,0.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9 days 09:27:40,26 days 15:21:11,0.0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13 days 05:00:36,26 days 04:31:54,0.0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2 days 20:58:23,12 days 02:41:21,0.0,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8 days 05:13:56,18 days 14:05:55,0.0,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22 days 04:38:58,23 days 11:01:02,0.0,delivered
99438,83c1379a015df1e13d02aae0204711ab,24 days 20:37:34,30 days 09:13:17,0.0,delivered
99439,11c177c8e97725db2631073c19f07b62,17 days 02:04:27,37 days 02:31:33,0.0,delivered


In [27]:
# check new dataframe and copy code carefully to `olist/order.py`

In [28]:
# YOUR CODE HERE

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, commit it to `olist/order.py` <br/>

🙏 Now, test it by running the following cell 👇 

In [29]:
# Test your code here
from olist.order import Order
Order().get_wait_time()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8 days 10:28:40,15 days 13:03:27,0.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13 days 18:46:08,19 days 03:18:23,0.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9 days 09:27:40,26 days 15:21:11,0.0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13 days 05:00:36,26 days 04:31:54,0.0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2 days 20:58:23,12 days 02:41:21,0.0,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8 days 05:13:56,18 days 14:05:55,0.0,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22 days 04:38:58,23 days 11:01:02,0.0,delivered
99438,83c1379a015df1e13d02aae0204711ab,24 days 20:37:34,30 days 09:13:17,0.0,delivered
99439,11c177c8e97725db2631073c19f07b62,17 days 02:04:27,37 days 02:31:33,0.0,delivered


### `get_review_score`
     Returns a DataFrame with:
        order_id, dim_is_five_star, dim_is_one_star, review_score

👉 Load the `reviews`

NameError: name 'data' is not defined

In [41]:
# YOUR CODE HERE
reviews = data['order_reviews']

👉 Let's create two functions `dim_five_star` and `dim_one_star`  
    We will apply them  `element_wise` to the `review_score` column in the next cell below.

In [47]:
# reviews[["order_id", "dim_is_five_star", "dim_is_one_star", "review_score"]
# reviews[["review_score"]].applymap(lambda x :1 if x==5 else 0)

# df.applymap(lambda x: 0 if x=='None' else 1)
# Out[33]: 


In [48]:
# reviews[["review_score"]].applymap(lambda d :5 if d==5 else 0)

In [52]:
# Fill in the functions below, which you will have to apply "element-wise" to each Series in the next cell below
# So as to create the 2 new columns requested 

def dim_five_star(d):
    return 5 if d==5 else 0

def dim_one_star(d):
    return 1 if d==1 else 0

👉 Use these functions to create two boolean features `dim_is_five_star` and `dim_is_one_star`

In [62]:
reviews["dim_is_five_star"] = reviews["review_score"].apply(dim_five_star) # --> Series([0, 1, 1, 0, 0, 1 ...])
reviews["dim_is_one_star"] = reviews["review_score"].apply(dim_one_star) # --> Series([0, 1, 1, 0, 0, 1 ...])


In [63]:
# def dim_five_star(d):
#     return d[["review_score"]].applymap(lambda x :1 if x==5 else 0, type=int)
# dim_five_star(d).dtypes

In [64]:
reviews[["order_id", "dim_is_five_star", "dim_is_one_star", "review_score"]]

,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,5,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,5,0,5
3,658677c97b385a9be170737859d3511b,5,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,5,0,5
99220,22ec9f0669f784db00fa86d035cf8602,5,0,5
99221,55d4004744368f5571d1f590031933e4,5,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


In [65]:
reviews[["order_id", "dim_is_five_star", "dim_is_one_star", "review_score"]].dtypes

order_id            object
dim_is_five_star     int64
dim_is_one_star      int64
review_score         int64
dtype: object

In [66]:
# Test your code here
from olist.order import Order
Order().get_review_score()

,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,5,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,5,0,5
3,658677c97b385a9be170737859d3511b,5,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,5,0,5
99220,22ec9f0669f784db00fa86d035cf8602,5,0,5
99221,55d4004744368f5571d1f590031933e4,5,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


#### Check your code

In [68]:
from nbresult import ChallengeResult

result = ChallengeResult('reviews',
    dim_five_star=dim_five_star(5),
    dim_not_five_star=dim_five_star(3),
    dim_one_star=dim_one_star(1),
    dim_not_one_star=dim_one_star(2)
)
result.write()
print(result.check())

============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /Users/shu/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/shu/Desktop/Lewagon/code/shiro101010101/data-challenges/04-Decision-Science/02-Statistical-Inference/01-Orders
plugins: dash-2.0.0, anyio-3.3.2
collecting ... collected 4 items

tests/test_reviews.py::TestReviews::test_dim_five_star PASSED            [ 25%]
tests/test_reviews.py::TestReviews::test_dim_not_five_star PASSED        [ 50%]
tests/test_reviews.py::TestReviews::test_dim_not_one_star PASSED         [ 75%]
tests/test_reviews.py::TestReviews::test_dim_one_star PASSED             [100%]

============================== 4 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/reviews.pickle

git commit -m 'Completed reviews step'

git push origin master


### `get_number_products`:
     Returns a DataFrame with:
        order_id, number_of_products (total number of products per order)

In [87]:
# YOUR CODE HERE
order_data=data['order_items']
order_data

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [86]:
# ~order_data["product_id"].duplicated().sum()

In [92]:
order_data[["order_id","order_item_id"]]

,order_id,order_item_id
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1
112646,fffcd46ef2263f404302a634eb57f7eb,1
112647,fffce4705a9662cd70adb13d4a31832d,1
112648,fffe18544ffabc95dfada21779c9644f,1


In [ ]:
# <your DataFrame>.rename(columns={'count':'Total_Numbers'})

In [107]:
num_order=order_data[["order_id","order_item_id"]].groupby("order_id")\
.count().rename(columns={"order_item_id":"number_of_products"})

In [110]:
# num_order.nunique()

In [112]:
num_order

,number_of_products
order_id,
00010242fe8c5a6d1ba2dd792cb16214,1
00018f77f2f0320c557190d7a144bdd3,1
000229ec398224ef6ca0657da4fc703e,1
00024acbcdf0a6daa1e931b038114c75,1
00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...
fffc94f6ce00a00581880bf54a75a037,1
fffcd46ef2263f404302a634eb57f7eb,1
fffce4705a9662cd70adb13d4a31832d,1


👉 Same routine: 
* check your dataframe, 
* commit your code to `olist/order.py`
* and check that it truly works.

In [115]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_number_products()

,number_of_products
order_id,
00010242fe8c5a6d1ba2dd792cb16214,1
00018f77f2f0320c557190d7a144bdd3,1
000229ec398224ef6ca0657da4fc703e,1
00024acbcdf0a6daa1e931b038114c75,1
00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...
fffc94f6ce00a00581880bf54a75a037,1
fffcd46ef2263f404302a634eb57f7eb,1
fffce4705a9662cd70adb13d4a31832d,1


### `get_number_sellers`:
     Returns a DataFrame with:
        order_id, number_of_sellers (total number of unique sellers per order)
        
<details>
    <summary>- <i>Hint</i> -</summary>

`pd.Series.nunique()`
</details>

In [152]:
# YOUR CODE HERE
sellers_data=data['order_items']
sellers_data

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [153]:
sellers_data=sellers_data[["order_id","seller_id"]]

In [164]:
sellers_data.groupby("order_id").count()

,seller_id
order_id,
00010242fe8c5a6d1ba2dd792cb16214,1
00018f77f2f0320c557190d7a144bdd3,1
000229ec398224ef6ca0657da4fc703e,1
00024acbcdf0a6daa1e931b038114c75,1
00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...
fffc94f6ce00a00581880bf54a75a037,1
fffcd46ef2263f404302a634eb57f7eb,1
fffce4705a9662cd70adb13d4a31832d,1


In [168]:
sellers_data.groupby("order_id").nunique().reset_index()

,order_id,seller_id
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1
98662,fffcd46ef2263f404302a634eb57f7eb,1
98663,fffce4705a9662cd70adb13d4a31832d,1
98664,fffe18544ffabc95dfada21779c9644f,1


In [169]:
sellers_data[["seller_id"]].nunique()

seller_id    3095
dtype: int64

In [170]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_number_sellers()

,order_id,seller_id
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1
98662,fffcd46ef2263f404302a634eb57f7eb,1
98663,fffce4705a9662cd70adb13d4a31832d,1
98664,fffe18544ffabc95dfada21779c9644f,1


### `get_price_and_freight`
     Returns a DataFrame with:
        order_id, price, freight_value

<details>
    <summary>- <i>Hint -</i></summary>

`pd.Series.agg()` allows you to apply one transformation method per column of your groupby object
</details>

In [197]:
# YOUR CODE HERE
data = olist.get_data()

In [198]:
# price_and_freight[["order_id","price", "freight_value"]]

In [199]:
price_freight = data['order_items'].groupby('order_id',
             as_index=False).agg({'price': 'sum',
                                  'freight_value': 'sum'})

In [200]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_price_and_freight()

,order_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14
...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,299.99,43.41
98662,fffcd46ef2263f404302a634eb57f7eb,350.00,36.53
98663,fffce4705a9662cd70adb13d4a31832d,99.90,16.95
98664,fffe18544ffabc95dfada21779c9644f,55.99,8.72


### `get_distance_seller_customer` 
**(OPTIONAL - Try  to code this function only after finishing today's challenges - Skip to next section)**

    Returns a Dataframe with:
        [order_id, distance_seller_customer] 
               (the distance in km between customer and seller)

💡Have a look at the `haversine_distance` formula we coded for you in the `olist.utils` module

In [ ]:
# YOUR CODE HERE

👀 Check your new dataframe and commit your code to olist/order.py when it works. 

In [ ]:
# YOUR CODE HERE

# Test your newly coded module

❓ Time to code `get_training_data` making use of your previous coded methods.

In [188]:
from olist.order import Order
from nbresult import ChallengeResult
data = Order().get_training_data()
result = ChallengeResult('training',
    shape=data.shape,
    columns=sorted(list(data.columns))
)
result.write()
print(result.check())

AttributeError: 'NoneType' object has no attribute 'shape'

🏁 Congratulations! 

⌛️ Commit and push your notebook before starting the next challenge.